In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]


In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(2)

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)

In [ ]:
from haystack.utils import fetch_archive_from_http

data_dir = "data/fine-tuning"


fetch_archive_from_http(
    url="https://drive.google.com/file/d/17Eav55_EGfAZGVDQONuvkfo4GkAp1DQZ/view?usp=sharing", output_dir=data_dir
)

False

In [ ]:
reader.train(data_dir=data_dir, train_filename="train.json", use_gpu=True, n_epochs=1, save_dir="my_model")

Train epoch 0/0 (Cur. train loss: 0.1958): 100%|██████████| 33/33 [06:46<00:00, 12.31s/it]


In [ ]:
new_reader = FARMReader(model_name_or_path="my_model")


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
from haystack.schema import Document

new_reader.predict(
    query="What is the gender of patient?", documents=[Document(content="male")]
)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.53 Batches/s]


{'query': 'What is the gender of patient?',
 'no_ans_gap': 10.974991083145142,
 'answers': [<Answer {'answer': 'male', 'type': 'extractive', 'score': 0.9893465042114258, 'context': 'male', 'offsets_in_document': [{'start': 0, 'end': 4}], 'offsets_in_context': [{'start': 0, 'end': 4}], 'document_ids': ['ceb2f9d98b5b18dcdf28379bbb748a34'], 'meta': {}}>]}

In [ ]:
#on test data

In [ ]:
# Import necessary libraries and modules
from haystack.telemetry import tutorial_running
from haystack.nodes import FARMReader
from haystack.schema import Document
from haystack.utils import fetch_archive_from_http, print_answers# Enable telemetry
tutorial_running(2)

# Fetch test data archive
data_dir = "data/fine-tuning"
fetch_archive_from_http(
    url="https://drive.google.com/uc?id=17Eav55_EGfAZGVDQONuvkfo4GkAp1DQZ", output_dir=data_dir
)

# Initialize the fine-tuned reader with your model
new_reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)

# Train the model on your own data and save it to “my_model”
new_reader.train(data_dir=data_dir, train_filename="test.json", use_gpu=True, n_epochs=1, save_dir="my_model")

# Test the fine-tuned model on a sample query
prediction = new_reader.predict(
    query="What is the gender of the patient?", documents=[Document(content="The patient is male.")]
)

# Print the answer
print_answers(prediction, details="all")


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.60 Batches/s]

'Query: What is the gender of the patient?'
'Answers:'
[   <Answer {'answer': 'male', 'type': 'extractive', 'score': 0.963762640953064, 'context': 'The patient is male.', 'offsets_in_document': [{'start': 15, 'end': 19}], 'offsets_in_context': [{'start': 15, 'end': 19}], 'document_ids': ['1cb87ed4684ff7292e391a6bfc992c0b'], 'meta': {}}>]


In [ ]:
# Extract the predicted answer
predicted_answer = prediction["answers"][0].answer

# Manually provide the ground truth answer from your test data (replace with the actual ground truth)
ground_truth_answer = "male"

# Calculate exact match and F1 score
exact_match = int(predicted_answer.lower() == ground_truth_answer.lower())
f1_score = 2 * (len(set(predicted_answer.lower().split()) & set(ground_truth_answer.lower().split()))) / (
        len(set(predicted_answer.lower().split())) + len(set(ground_truth_answer.lower().split()))
)

# Print the results
print(f"Predicted Answer: {predicted_answer}")
print(f"Ground Truth Answer: {ground_truth_answer}")
print(f"Exact Match: {exact_match}")
print(f"F1 Score: {f1_score}")


Predicted Answer: male
Ground Truth Answer: male
Exact Match: 1
F1 Score: 1.0
